# Hull–White (1F) Extended Vasicek Model Bond Option Pricing 

This notebook builds three input curves (flat, pillar zeros, bootstrapped), prices **European options on zero‑coupon bonds** under the **Hull–White 1‑factor model**, and compares to a Black (Black‑76) pricer. 

## Imports & utilities


In [1]:
import numpy as np
from math import exp, log, sqrt, erf
from dataclasses import dataclass
from typing import List, Tuple, Optional

In [23]:
# Utilities
def Phi(x: float) -> float:
    """Standard normal CDF."""
    return 0.5 * (1.0 + erf(x / sqrt(2.0)))

## Define piecewise-linear zero curve
- Defines a piecewise‑linear continuous‑compounded zero curve and provides `discount(T)` and simple curve bumps.
- Builds three curves: flat, pillar zeros, and bootstrapped; these feed the pricers.

In [25]:
# Piecewise-linear zero curve
#hold a yield curve and give discount factors for any maturity T

@dataclass
class PiecewiseLinearZeroCurve:
    """
    Continuous-compounded zero curve z(T), linearly interpolated in T (years).
    Stores pillars (T_i, z_i). Discount: P(0,T) = exp(-z(T)*T).
    """
    maturities: np.ndarray
    zeros: np.ndarray

    def __post_init__(self):
        mats = np.array(self.maturities, dtype=float)
        zs   = np.array(self.zeros, dtype=float)
        idx  = np.argsort(mats)
        self.maturities = mats[idx]
        self.zeros      = zs[idx]

    # ---- Constructors ----
    @classmethod
    def from_flat(cls, flat_cc_rate: float, T_max: float = 30.0):
        mats  = np.array([0.0, T_max], dtype=float)
        zeros = np.array([flat_cc_rate, flat_cc_rate], dtype=float)
        return cls(mats, zeros)

    @classmethod
    def from_zeros(cls, pillars_T: List[float], zeros_cc: List[float]):
        return cls(np.array(pillars_T, float), np.array(zeros_cc, float))

    @classmethod
    def from_discounts(cls, pillars_T: List[float], discounts: List[float]):
        T = np.array(pillars_T, dtype=float)
        P = np.clip(np.array(discounts, dtype=float), 1e-12, 1.0)
        z = np.zeros_like(T)
        mask = T > 0
        z[mask] = -np.log(P[mask]) / T[mask]
        if not mask.all():
            # At T=0 set z to the first positive-maturity zero
            z[~mask] = z[mask][0]
        return cls(T, z)

    def z(self, T: float) -> float:
        mats, zs = self.maturities, self.zeros
        if T <= mats[0]:  return zs[0]
        if T >= mats[-1]: return zs[-1]
        i = np.searchsorted(mats, T) - 1
        T0, T1 = mats[i], mats[i+1]
        z0, z1 = zs[i], zs[i+1]
        w = (T - T0) / (T1 - T0)
        return (1 - w) * z0 + w * z1

    def discount(self, T: float) -> float:
        return exp(-self.z(T) * T)

    # Simple parallel and key-rate bumps
    def bumped_parallel(self, bp: float):
        bumped = self.zeros + bp / 10000.0
        return PiecewiseLinearZeroCurve(self.maturities.copy(), bumped)

    def bumped_keyrate(self, key_T: float, bp: float):
        mats = self.maturities.copy()
        zs   = self.zeros.copy()
        idx  = int(np.argmin(np.abs(mats - key_T)))
        zs[idx] += bp / 10000.0
        return PiecewiseLinearZeroCurve(mats, zs)


## Bootstrap yield curve from deposits & par swaps
- Bootstraps discount factors from a 1Y deposit and annual par swap quotes using par‑swap parity.

In [26]:
# Toy bootstrap (deposits + annual par swaps)

def bootstrap_discounts_from_deposits_swaps(
    dep_points: Optional[List[Tuple[float, float]]] = None,
    swap_points: Optional[List[Tuple[int, float]]] = None,
    accruals: Optional[List[float]] = None,
):
    """
    Bootstrap P(0,T) from:
      - deposits: (T_years, r_simple)  -> P = 1 / (1 + r_simple * T)
      - par swaps: (T_n_years, s_n) with annual fixed leg
    For simplicity: annual accruals alpha_i = 1 unless provided via 'accruals'.
    """
    dep_points  = dep_points  or []
    swap_points = swap_points or []
    disc = {}

    # deposits
    for T, r_simple in dep_points:
        disc[int(round(T))] = 1.0 / (1.0 + float(r_simple) * float(T))

    # swaps (sorted by maturity)
    swap_points = sorted(swap_points, key=lambda x: x[0])
    for Tn, s in swap_points:
        s = float(s)
        n = int(Tn)
        # ensure earlier pillars exist
        annuity = 0.0
        for i in range(1, n):
            if i not in disc:
                raise ValueError(f"Missing discount for {i}y; add a deposit or earlier swap.")
            alpha_i = 1.0 if accruals is None else accruals[i-1]
            annuity += alpha_i * disc[i]
        alpha_n = 1.0 if accruals is None else accruals[n-1]
        # Par swap parity (fixed annual):
        # 1 - P(0,Tn) = s * sum_{i=1..n} alpha_i P(0,Ti)
        # => P(0,Tn) * (1 + s * alpha_n) = 1 - s * sum_{i=1..n-1} alpha_i P(0,Ti)
        P_Tn = (1.0 - s * annuity) / (1.0 + s * alpha_n)
        if P_Tn <= 0:
            raise ValueError(f"Negative discount at {n}y; check inputs.")
        disc[n] = P_Tn

    # Assemble pillars
    pillars = sorted(disc.keys())
    discounts = [disc[k] for k in pillars]
    return PiecewiseLinearZeroCurve.from_discounts(pillars, discounts)


## Define Hull–White (1F) model & bond-option pricer
- Implements Hull–White 1‑factor bond‑option pricing: computes bond price lognormal volatility σ_P and Black‑style price.

In [27]:

# Hull–White 1-factor (Extended Visicek Model) (bond option)
@dataclass
class HullWhite1F:
    a: float
    sigma: float
    curve: PiecewiseLinearZeroCurve

    def B(self, tau: float) -> float:
        """HW B(t,T) = (1 - e^{-a * tau}) / a, depends only on tau = T - t."""
        a = self.a
        return (1.0 - np.exp(-a * tau)) / a

    def sigma_P(self, t: float, S: float, T: float) -> float:
        """
        Bond price lognormal vol over [t,S]:
        sigma_P^2 = sigma^2 * ((1 - e^{-a(T-S)})/a)^2 * ((1 - e^{-2a(S-t)})/(2a))
        """
        a, sigma = self.a, self.sigma
        term1 = (1.0 - np.exp(-a * (T - S))) / a
        term2 = (1.0 - np.exp(-2.0 * a * (S - t))) / (2.0 * a)
        return sigma * term1 * sqrt(term2)

    def zcb_option(self, S: float, T: float, K: float, t: float = 0.0, call: bool = True) -> float:
        """
        Closed-form option on zero-coupon bond under HW1f at time t (default 0).
        This is identical in form to Black with vol = sigma_P(t,S,T).
        """
        P_tS = self.curve.discount(S)
        P_tT = self.curve.discount(T)
        sp   = self.sigma_P(t, S, T)
        if sp <= 0:
            intrinsic = P_tT - K * P_tS
            return max(intrinsic, 0.0) if call else max(-intrinsic, 0.0)
        d1 = (log(P_tT / (K * P_tS)) / sp) + 0.5 * sp
        d2 = d1 - sp
        if call:
            return P_tT * Phi(d1) - K * P_tS * Phi(d2)
        else:
            return K * P_tS * Phi(-d2) - P_tT * Phi(-d1)

    def call(self, S: float, T: float, K: float, t: float = 0.0) -> float:
        return self.zcb_option(S, T, K, t, True)

    def put(self,  S: float, T: float, K: float, t: float = 0.0) -> float:
        return self.zcb_option(S, T, K, t, False)


## Define Black (Black-76) bond option pricer
- Implements Black‑76 bond option formula to compare against HW using a chosen lognormal volatility.

In [28]:
# Black (Black-76) bond option pricer
class BlackBondOption:
    @staticmethod
    def price(P_tS: float, P_tT: float, K: float, sigma_ln: float, call: bool = True) -> float:
        """
        Black-style price for option on a zero-coupon bond at time t (use t=0 inputs).
        sigma_ln is the *lognormal* volatility of the bond price over [t,S].
        """
        if sigma_ln <= 0:
            intrinsic = P_tT - K * P_tS
            return max(intrinsic, 0.0) if call else max(-intrinsic, 0.0)
        d1 = (log(P_tT / (K * P_tS)) / sigma_ln) + 0.5 * sigma_ln
        d2 = d1 - sigma_ln
        if call:
            return P_tT * Phi(d1) - K * P_tS * Phi(d2)
        else:
            return K * P_tS * Phi(-d2) - P_tT * Phi(-d1)


## Bond option price
- Bootstraps discount factors from a 1Y deposit and annual par swap quotes using par‑swap parity.
- Implements Black‑76 bond option formula to compare against HW using a chosen lognormal volatility.
- Builds three curves: flat, pillar zeros, and bootstrapped; these feed the pricers.
- Runs a comparison table (HW vs Black) for each curve; uses K=0.8×P(0,T) or ATM as specified.

In [30]:
def compare_all_three_curves():
    #  Build three curves 
    # (A) Flat 3% cc
    curve_flat = PiecewiseLinearZeroCurve.from_flat(0.03)

    # (B) Pillar zeros (continuous compounding)
    pillars = [0.5, 1, 2, 3, 5, 7, 10]
    zeros   = [0.028, 0.029, 0.030, 0.031, 0.032, 0.033, 0.034]
    curve_zeros = PiecewiseLinearZeroCurve.from_zeros(pillars, zeros)

    # (C) Bootstrap from deposits + annual par swaps (toy numbers)
    dep_points  = [(1, 0.0300)]              # 1Y deposit, 3.00% simple
    swap_points = [(2, 0.0310), (3, 0.0320), (4, 0.0325), (5, 0.0330)]  # par swap rates
    curve_boot  = bootstrap_discounts_from_deposits_swaps(dep_points, swap_points)

    curves = {
        "Flat 3%": curve_flat,
        "Pillar Zeros": curve_zeros,
        "Bootstrap (dep+swaps)": curve_boot
    }
    
    # HW parameters & trade 
    a, sigma = 0.05, 0.010
    S, T = 2.0, 5.0
    # Strike as 80% of P(0,T) on *each* curve (so the moneyness is consistent per curve)
    rel_strike = 0.80

    
    print("=== HW1f vs Black (Black-76) for a ZCB Option ===")
    print(f"HW params: a={a:.4f}, sigma={sigma:.4f}, Option expiry S={S}y, Bond maturity T={T}y")
    print("Note: Black price equals HW if sigma_ln = HW sigma_P(t,S,T).")
    print("-" * 86)
    print(f"{'Curve':26s} {'P(0,S)':>10s} {'P(0,T)':>10s} {'K':>10s} {'HW Call':>12s} {'HW Put':>12s} {'Black@HWvol':>12s} {'Black@10%vol':>12s}")

    # price 
    for name, curve in curves.items():
        hw   = HullWhite1F(a, sigma, curve)
        P0S  = curve.discount(S)
        P0T  = curve.discount(T)
        K    = rel_strike * P0T

        
        # HW prices
        call_hw = hw.call(S, T, K, t=0.0)
        put_hw  = hw.put(S,  T, K, t=0.0)

        # Black with HW-equivalent vol (should match HW)
        sigma_hw = hw.sigma_P(0.0, S, T)
        call_black_hw = BlackBondOption.price(P0S, P0T, K, sigma_hw, call=True)

        # Black with an arbitrary 10% lognormal vol (to show model-risk delta)
        call_black_10 = BlackBondOption.price(P0S, P0T, K, 0.10, call=True)

        print(f"{name:26s} {P0S:10.6f} {P0T:10.6f} {K:10.6f} {call_hw:12.6f} {put_hw:12.6f} {call_black_hw:12.6f} {call_black_10:12.6f}")
# Run
if __name__ == "__main__":
    compare_all_three_curves()

=== HW1f vs Black (Black-76) for a ZCB Option ===
HW params: a=0.0500, sigma=0.0100, Option expiry S=2.0y, Bond maturity T=5.0y
Note: Black price equals HW if sigma_ln = HW sigma_P(t,S,T).
--------------------------------------------------------------------------------------
Curve                          P(0,S)     P(0,T)          K      HW Call       HW Put  Black@HWvol Black@10%vol
Flat 3%                      0.941765   0.860708   0.688566     0.212241     0.000000     0.212241     0.212292
Pillar Zeros                 0.941765   0.852144   0.681715     0.210129     0.000000     0.210129     0.210179
Bootstrap (dep+swaps)        0.940740   0.849821   0.679857     0.210253     0.000000     0.210253     0.210301


We printed comparison table of HW1f vs Black for bond options price across curves. The Hull-White model price matches Black@σ_HW price.

## ATM-forward pricing on flat curve and  parity check
-Builds a flat 3% zero curve and sets HW1f parameters $a=0.05, \sigma =0.01$ for an option expires in $S= 2Y$ 
 on a zero-coupon bond   expiring in $T=5Y$
- Computes $P(0,S)$ and $P(0,T)$ and the ATM-forward strike $K_{ATM}=\frac{P(0,S)}{P(0,T)}$
- Computes the HW bond-price lognormal volatility $\sigma_p$ over $[0,S]$
- Price options using both Hull-White(Black with $\sigma_p$)  and Black-Scholes methods
- Check the put-call parity 



In [35]:
# curve (flat 3%)
curve_flat = PiecewiseLinearZeroCurve.from_flat(0.03)

#model params & trade 
a, sigma = 0.05, 0.010
S, T = 2.0, 5.0
hw = HullWhite1F(a, sigma, curve_flat)

#discounts & ATM-forward strike 
P0S = curve_flat.discount(S)
P0T = curve_flat.discount(T)
K_atm = P0T / P0S

# HW bond-price vol over [0,S] 
sig_hw = hw.sigma_P(0.0, S, T)

# Prices (HW = Black with HW vol)
call_hw_atm  = BlackBondOption.price(P0S, P0T, K_atm, sig_hw, call=True)
call_blk_10  = BlackBondOption.price(P0S, P0T, K_atm, 0.10,  call=True)
call_hw_formula = hw.zcb_option(S, T, K_atm, t=0.0, call=True)

print(f"P(0,S)={P0S:.6f}  P(0,T)={P0T:.6f}  K_atm={K_atm:.6f}  sig_hw={sig_hw:.6f}")
print(f"Call@HW={call_hw_formula:.6f} Call@HWvol={call_hw_atm:.6f}   Call@10%vol={call_blk_10:.6f}")

# put-call parity 
put_hw_atm = BlackBondOption.price(P0S, P0T, K_atm, sig_hw, call=False)
print("Parity check: Call - Put =", call_hw_atm - put_hw_atm, "  vs  P(0,T) - K*P(0,S) =", P0T - K_atm*P0S)





P(0,S)=0.941765  P(0,T)=0.860708  K_atm=0.913931  sig_hw=0.037508
Call@HW=0.012878 Call@HWvol=0.012878   Call@10%vol=0.034323
Parity check: Call - Put = 0.0   vs  P(0,T) - K*P(0,S) = 0.0


Using Hull–White (1f), the bond-price volatility over 2y is $\sigma_p=3.75 \%$. The ATM-forward call on the 5Y zero-coupon bond is $0.012878$ per $\$1$ notional. Black-76 with the same $\sigma_p$ gives the same price (sanity check passed). With an ad-hoc $10\%$ vol, the call jumps to $0.034323 (+0.02145)$, showing strong vega / volatility model risk. Put–call parity holds exactly at ATM (Call − Put = 0
P(0,T)−KP(0,S)=0), confirming internal consistency.

## ATM-forward comparison across three curves and volatility sensitivity 

In [36]:
# --- rebuild the three curves (same as in your demo) ---
curve_flat   = PiecewiseLinearZeroCurve.from_flat(0.03)

pillars = [0.5, 1, 2, 3, 5, 7, 10]
zeros   = [0.028, 0.029, 0.030, 0.031, 0.032, 0.033, 0.034]
curve_zeros  = PiecewiseLinearZeroCurve.from_zeros(pillars, zeros)

dep_points  = [(1, 0.0300)]
swap_points = [(2, 0.0310), (3, 0.0320), (4, 0.0325), (5, 0.0330)]  
curve_boot   = bootstrap_discounts_from_deposits_swaps(dep_points, swap_points)

curves = [("Flat 3%", curve_flat), ("Pillar Zeros", curve_zeros), ("Bootstrap (dep+swaps)", curve_boot)]

# --- HW params & trade ---
a, sigma = 0.05, 0.010
S, T = 2.0, 5.0

print(f"{'Curve':26s} {'K_ATM':>12s} {'sig_hw':>10s} {'Call@HWvol':>14s} {'Call@10%vol':>14s}")
for label, curve in curves:
    hw   = HullWhite1F(a, sigma, curve)
    P0S  = curve.discount(S)
    P0T  = curve.discount(T)
    K_atm = P0T / P0S
    sig_hw = hw.sigma_P(0.0, S, T)
    call_hw_atm = BlackBondOption.price(P0S, P0T, K_atm, sig_hw, call=True)
    call_blk_10 = BlackBondOption.price(P0S, P0T, K_atm, 0.10,  call=True)
    print(f"{label:26s} {K_atm:12.6f} {sig_hw:10.6f} {call_hw_atm:14.6f} {call_blk_10:14.6f}")


Curve                             K_ATM     sig_hw     Call@HWvol    Call@10%vol
Flat 3%                        0.913931   0.037508       0.012878       0.034323
Pillar Zeros                   0.904837   0.037508       0.012750       0.033981
Bootstrap (dep+swaps)          0.903354   0.037508       0.012715       0.033889


- $\sigma_{hw}$ is identical $0.037508$ for all rows
- Different curve constructions give slightly different discounts, so the ATM strike moves
- switching the volatility assumption from the HW-implied $~3.75\%$ to an ad-hoc $10\%$ roughly triples the price — so volatility choice dominates curve construction for this option.

## ATM-forward parameter sensitivity on flat curve ($±20\%$ in $a$ and $\sigma$)

In [37]:
# Using the flat curve above:
def price_atm(curve, a, sigma, S, T):
    hw   = HullWhite1F(a, sigma, curve)
    P0S  = curve.discount(S)
    P0T  = curve.discount(T)
    K_atm = P0T / P0S
    sig_hw = hw.sigma_P(0.0, S, T)
    return BlackBondOption.price(P0S, P0T, K_atm, sig_hw, call=True)

base = price_atm(curve_flat, a, sigma, S, T)
p_a_up = price_atm(curve_flat, a*1.2, sigma, S, T)
p_a_dn = price_atm(curve_flat, a*0.8, sigma, S, T)
p_s_up = price_atm(curve_flat, a, sigma*1.2, S, T)
p_s_dn = price_atm(curve_flat, a, sigma*0.8, S, T)

print(f"Base={base:.6f} | a +20%: {p_a_up-base:+.6f}  a -20%: {p_a_dn-base:+.6f} | sigma +20%: {p_s_up-base:+.6f}  sigma -20%: {p_s_dn-base:+.6f}")


Base=0.012878 | a +20%: -0.000308  a -20%: +0.000318 | sigma +20%: +0.002575  sigma -20%: -0.002575


- Base price $= 0.012878$
- $a +20\%$ ->$−0.000308$ (≈$ −2.4\%$ of base) and $a −20\%$ → $+0.000318$ ($≈ +2.5\%)$
- Higher mean reversion lowers long-dated bond-price and vice-versa, but the effect is modest 
- $\sigma +20\%$ → $+0.002575$ ($≈ +20.0\%$ of base), and $\sigma -20\% -> $ −0.002575$ ($+20\%), dominant effect
- At ATM, the HW1f call is far more sensitive to volatility ($\sigma$) than to mean reversion($a$)


### Summary

As theory predicts, Hull–White and Black-76 give the same option price when Black uses the volatility implied by Hull–White. At the money, the option was worth about 1.29 cents per $1 notional with the model-implied volatility; forcing a higher 10% volatility lifted the price to about 3.43 cents, showing that volatility choice is the biggest driver of value. Changing mean-reversion had a much smaller effect than changing volatility, and switching among the three curve constructions only moved the price by about 1%. Put–call parity held exactly, and deep in-the-money/ out-of-the-money behavior matched intuition (e.g., very low strike ⇒ put near zero). Overall, the project demonstrates a correct implementation, clear sanity checks, and a model-risk takeaway: volatility assumptions matter far more than curve construction or mean-reversion for this trade.